In [5]:
import socket, os, struct, datetime, codecs, time

def checksum(source_string):
    sum = 0
    countTo = (len(source_string)/2)*2
    count = 0
    while count<countTo:
        thisVal = source_string[count + 1]*256 + source_string[count]
        sum = sum + thisVal
        sum = sum & 0xffffffff
        count = count + 2
    if countTo<len(source_string):
        sum = sum + source_string[len(source_string) - 1]
        sum = sum & 0xffffffff
    sum = (sum >> 16)  +  (sum & 0xffff)
    sum = sum + (sum >> 16)
    answer = ~sum
    answer = answer & 0xffff
    return answer

def ping(dest, tout, attempts):
    #Retreive IP address 
    try:
        ip = socket.gethostbyname(dest)
    except socket.gaierror:
        print("Invalid address.")
        return
    
    loss = receive = mint = maxt = avg = std = 0
    rttlist = []
    
    #Socket creation
    icmp = socket.getprotobyname("icmp")
    clientsocket = socket.socket(socket.AF_INET, socket.SOCK_RAW, icmp)
    
    clientsocket.connect((ip, 7000))
    print("\nPinging " + dest + " [" + ip + "] " + "...")

    for x in range(0, attempts):
        #Packet creation
        ICMP_TYPE = 8
        ICMP_CODE = 0
        ID = os.getpid()&0xFFFF
        CHECKSUM = 0
        SEQUENCE = 1
        header = struct.pack("bbHHh", ICMP_TYPE, ICMP_CODE, CHECKSUM, ID, SEQUENCE)
        t = datetime.datetime.now()
        data = bytes(str(t),"utf-8")
        CHECKSUM = checksum(header + data)
        header = struct.pack("bbHHh", ICMP_TYPE, ICMP_CODE, CHECKSUM, ID, SEQUENCE)
        packet = header + data

        #Pings 'attempts' times, with timeout set to 'tout' seconds
        try:
            clientsocket.send(packet)
            clientsocket.settimeout(tout)
            reply = clientsocket.recv(2048)
            replylist = reply.decode('utf-8', 'ignore').split(' ')
            t = replylist.pop()

            #Time when reply arrives
            current = str(datetime.datetime.now())
            chrs, cmins, csecs = current.split(':')
            
            #Extract time
            hrs, mins, secs = t.split(':')
            
            #Prevents the receiving of the previous packet after its timeout
            if((float(csecs) - float(secs)) > tout):
                raise socket.timeout
                
            receive += 1
            rtt = '%.3f'%((float(csecs) - float(secs)) * 1000)
            rttlist.append(rtt)
            print("Reply from " + addr + " [" + ip + "] : " + "time = " + str(rtt) + "ms")

        except socket.timeout:
            print("Timeout within " + str(tout) + " secs.")
    
    #Calculates packet loss
    if(attempts != 0):
        loss = (attempts - receive) * 100 / attempts
        
    #Finds the average of RTT
    for x in rttlist:
        avg += float(x)
    
    if(attempts != (attempts - receive)):
        avg = '%.3f'%(avg / receive) 
        
    #Finds the standard deviation of RTT
    for x in rttlist:
        std += (float(x) - float(avg)) ** 2
        
    if(attempts != (attempts - receive)):
        std = '%.3f'%(std / receive) ** 0.5
      
    #Finds minimum RTT
    if(len(rttlist) > 0):
        mint = rttlist[0]
        maxt = rttlist[0]
        
    for x in rttlist:
        if(float(x) < float(mint)):
            mint = x
            
    #Finds maximum RTT
    for x in rttlist:
        if(float(x) > float(maxt)):
            maxt = x
    
    print("\nPing statistics for [" + ip + "] : ")
    print("Packets: Sent = " + str(attempts) + ", Received = " + str(receive) + ", Lost = " + str(attempts - receive) + " (" + str(loss) +  "% loss)")
    print("Round trip times : Minimum = " + str(mint) + "ms, Maximum = " + str(maxt) + "ms, Average = " + str(avg) + "ms, Standard Deviation = " + str(std) + "ms")
    clientsocket.shutdown(socket.SHUT_RDWR)
    clientsocket.close()
#main
run = 1
while (run == 1):
    print("Enter address : ", end = ' ')
    addr = input()
    if(addr == "q"):
        run = 0
        print("Exiting program.")
    else:
        print("Enter timeout in seconds : ", end = ' ')
        timeout = float(input())
        print("Enter number of attempts : ", end = ' ')
        tries = int(input())
        ping(addr, timeout, tries)
        print("\n")




Enter address :  google.com
Enter timeout in seconds :  4
Enter number of attempts :  4

Pinging google.com [172.217.11.14] ...
Reply from google.com [172.217.11.14] : time = 22.074ms
Reply from google.com [172.217.11.14] : time = 20.764ms
Reply from google.com [172.217.11.14] : time = 15.629ms
Reply from google.com [172.217.11.14] : time = 14.404ms

Ping statistics for [172.217.11.14] : 
Packets: Sent = 4, Received = 4, Lost = 0 (0.0% loss)
Round trip times : Minimum = 14.404ms, Maximum = 22.074ms, Average = 18.218ms, Standard Deviation = 3.263ms


Enter address :  q
Exiting program.


#### 